In [914]:
import json
import numpy as np
import pandas as pd
import lxml

# Import SAP data previously mapped for ZA1

15Dec_D365_orders_ZA1.csv

In [915]:
df5 = pd.read_csv("./data/ZA1/15Dec_D365_orders_ZA1.csv", low_memory=False)

In [916]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 871832 entries, 0 to 871831
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   ActivityType              871832 non-null  object
 1   Order Number              871832 non-null  int64 
 2   Material_Description      871832 non-null  object
 3   BaseDate                  871832 non-null  object
 4   BaseHour                  871832 non-null  int64 
 5   Cases                     871832 non-null  int64 
 6   Bill I                    867796 non-null  object
 7   Source Channel            871832 non-null  object
 8   Order Category            871832 non-null  object
 9   D365_ItemNo               871832 non-null  object
 10  Material No               871832 non-null  object
 11  MOD                       871832 non-null  int64 
 12  site_id                   871832 non-null  object
 13  WAREHOUSELOCATIONID       871832 non-null  object
 14  D365

# Import Products, Customers and Warehouses for the Legal Entity

In [917]:
LE = 'UG1'

In [918]:
if LE == 'NA1':
    LE_Products = pd.read_csv("./data/NA1/Group1Products_NA1.csv", delimiter="\t")

if LE == 'MZ1':
    LE_Products = pd.read_csv("./data/MZ1/Group1Products_MZ1.csv", delimiter=",")

if LE == 'UG1':
    LE_Products = pd.read_csv("./data/UG1/Group1Products_UG1.csv", delimiter=",")
    

In [919]:
LE_Products

,D365_ItemNo_LE,Material_Description_LE
0,5141,SCH NOVIDA PINEAPPLE 500ML 12 S/W NP
1,10208,SCH NOVIDA P/APPLE ZERO 350ML 12 S/W NP
2,1271,COKE 500ML 12 S/W NP
3,2062,SPRITE 500ML 20 RB
4,5157,SCH NOVIDA PINEAPPLE BREEZ 300ML 24 RB
...,...,...
76,5465,SCH TONIC 300ML 24 RB
77,7477,RWENZORI WATER 500ML (24 BOX) NP
78,11165,SPRITE ZERO 330ML 12 S/W NP
79,10811,MM REFRESH MANGO 280ML 12 S/W


In [920]:
if LE == 'NA':
    LE_Customers = pd.read_csv("./data/NA1/NA1_Customers.csv", delimiter="\t")
    LE_Customers.rename(columns={'Validated Customer Accounts': 'D365_Cust_No_LE'}, inplace=True)
 

if LE == 'MZ':
    LE_Customers = pd.read_csv("./data/MZ1/PERF01_MZ1_Customers.csv", delimiter="\t")    
    LE_Customers.rename(columns={'CustomerAccount': 'D365_Cust_No_LE'}, inplace=True)
    
if LE == 'UG':
    LE_Customers = pd.read_csv("./data/UG1/IPT3 UG1- Customers.csv", delimiter=",")    
    LE_Customers.rename(columns={'CustomerAccount': 'D365_Cust_No_LE'}, inplace=True)
    LE_Customers = LE_Customers[['D365_Cust_No_LE']].copy()
    # Name

In [921]:
LE_Customers

,D365_Cust_No_LE,Index
0,11547352,0
1,11548191,1
2,11549879,2
3,11557561,3
4,11559624,4
...,...,...
165,11609410,165
166,11609412,166
167,11609433,167
168,11609463,168


In [922]:
if LE == 'NA':
    LE_Warehouses = pd.read_csv("./data/NA1/NA1_Validated  IPT3 Warehouse Export  NA1 Warehouses.csv", delimiter=",")
    LE_Warehouses.rename(columns={'D365_Del_Location': 'D365_Del_Loc_LE'}, inplace=True)
    LE_Warehouses = LE_Warehouses[['D365_Del_Loc_LE']].copy()
    #D365_Del_Location

if LE == 'MZ':
    LE_Warehouses = pd.read_csv("./data/MZ1/IPT3 MZ1 PERF01 Warehouse.csv", delimiter=",")
    LE_Warehouses.rename(columns={'D365DELIVERYLOCATION': 'D365_Del_Loc_LE'}, inplace=True)
    LE_Warehouses = LE_Warehouses[['D365_Del_Loc_LE']].copy()
    #D365DELIVERYLOCATION

if LE == 'UG':
    LE_Warehouses = pd.read_csv("./data/UG1/IPT3 WMS UG1-Warehouses.csv", delimiter=",")    
    LE_Warehouses.rename(columns={'D365_Del_Loc': 'D365_Del_Loc_LE'}, inplace=True)
    LE_Warehouses = LE_Warehouses[['D365_Del_Loc_LE']].copy()
    #D365_Del_Loc

In [923]:
LE_Warehouses

,D365_Del_Loc_LE,Index
0,MZ1-MZ010B,0
1,MZ1-MZ010Q,1
2,MZ1-MZ011B,2
3,MZ1-MZ011Q,3
4,MZ1-MZ012B,4
5,MZ1-MZ012Q,5
6,MZ1-MZ013B,6


## Get the top n warehouses and filter the dataframe for those

In [924]:
# Group by site_id and count unique Order Numbers
#warehouse_order_counts = df5.groupby('site_id')['Order Number'].nunique().reset_index(name='OrderLinesCount')
warehouse_order_counts = df5.groupby('D365_Del_Loc')['Order Number'].nunique().reset_index(name='OrderLinesCount')
#D365_Del_Loc 

# Sort by OrderLinesCount in descending order and get the top 9 warehouses
#top_warehouses = warehouse_order_counts.sort_values(by='OrderLinesCount', ascending=False).head(len(LE_Warehouses))
top_warehouses = warehouse_order_counts.sort_values(by='OrderLinesCount', ascending=False)

#top_warehouses['Grouping'] = (top_warehouses.index % 7) + 1


In [925]:
top_warehouses.reset_index(inplace=True, drop=True)

In [926]:
top_warehouses['Grouping'] = (top_warehouses.index % len(LE_Warehouses))

In [927]:
top_warehouses

,D365_Del_Loc,OrderLinesCount,Grouping
0,ZA1-ZA014B,1113,0
1,ZA1-ZA016G,983,1
2,ZA1-ZA516B,951,2
3,ZA1-ZA020B,922,3
4,ZA1-ZA032B,921,4
...,...,...,...
82,ZA1-ZA034B,16,5
83,ZA1-ZA200B,13,6
84,ZA1-ZA565B,10,0
85,ZA1-ZA039B,5,1


In [928]:
# Filter the original DataFrame to only include rows where site_id is in the list of top warehouses
#top_warehouse_ids = top_warehouses['D365_Del_Loc'].tolist()
#df5_W = df5[df5['D365_Del_Loc'].isin(top_warehouse_ids)]
df5_W = df5.copy()

In [929]:
top_warehouses.reset_index(inplace=True, drop=True)

## Get the top n customers and filter the dataframe for those

In [930]:
# Group by D365_Cust_No and count unique Order Numbers
customer_order_counts = df5_W.groupby('D365_Cust_No')['Order Number'].nunique().reset_index(name='OrderLinesCount')

# Sort by OrderLinesCount in descending order and get the top n customers
top_customers = customer_order_counts.sort_values(by='OrderLinesCount', ascending=False).head(len(LE_Customers))

# Display the result
print(top_customers)

      D365_Cust_No  OrderLinesCount
1605      10017885               73
128       10004521               63
1607      10017888               57
137       10004547               56
1603      10017883               47
...            ...              ...
4059      10040252               13
651       10006715               13
839       10007672               13
4415      10043091               13
6386      10115944               13

[170 rows x 2 columns]


In [931]:
top_customer_ids = top_customers['D365_Cust_No'].tolist()
df5_W_C = df5_W[df5_W['D365_Cust_No'].isin(top_customer_ids)]

In [932]:
top_customers.reset_index(inplace=True, drop=True)

## Get the top n products and filter the dataframe for those

In [933]:
product_order_counts = df5_W_C.groupby('D365_ItemNo')['Order Number'].nunique().reset_index(name='OrderLinesCount')


top_products = product_order_counts.sort_values(by='OrderLinesCount', ascending=False).head(len(LE_Products))


print(top_products)

    D365_ItemNo  OrderLinesCount
65        10207              914
272   CB0200017              688
78         1040              643
57        10194              636
202        6540              582
..          ...              ...
47        10170               64
267        7947               63
144        4765               61
241        7534               60
69        10234               59

[81 rows x 2 columns]


In [934]:
top_product_ids = top_products['D365_ItemNo'].tolist()
df5_W_C_P = df5_W_C[df5_W_C['D365_ItemNo'].isin(top_product_ids)]

In [935]:
top_products.reset_index(inplace=True, drop=True)

## Now replace warehouses, customers and products for the data in the new legal entity
Extend the dataframes produced above, top_warehouses, top_customers and top_products by adding a column (for the same number of rows).  The new column contains the corresponding data for the new legal entity.

### Create Products lookup table

In [936]:
# Create a new column 'Index' in both dataframes with current index values
top_products['Index'] = top_products.index
LE_Products['Index'] = LE_Products.index

In [937]:
if len(LE_Products) != len(top_products):
    raise SystemExit("Mismatch in number of products")

In [938]:
LE_Products

,D365_ItemNo_LE,Material_Description_LE,Index
0,5141,SCH NOVIDA PINEAPPLE 500ML 12 S/W NP,0
1,10208,SCH NOVIDA P/APPLE ZERO 350ML 12 S/W NP,1
2,1271,COKE 500ML 12 S/W NP,2
3,2062,SPRITE 500ML 20 RB,3
4,5157,SCH NOVIDA PINEAPPLE BREEZ 300ML 24 RB,4
...,...,...,...
76,5465,SCH TONIC 300ML 24 RB,76
77,7477,RWENZORI WATER 500ML (24 BOX) NP,77
78,11165,SPRITE ZERO 330ML 12 S/W NP,78
79,10811,MM REFRESH MANGO 280ML 12 S/W,79


In [939]:
# Merge based on Index
top_products = pd.merge(
    top_products,
    LE_Products,
    how="left",
    on="Index",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [940]:
top_products.drop(columns={'OrderLinesCount', 'Index'}, inplace=True, axis=1)

In [941]:
top_products

,D365_ItemNo,D365_ItemNo_LE,Material_Description_LE
0,10207,5141,SCH NOVIDA PINEAPPLE 500ML 12 S/W NP
1,CB0200017,10208,SCH NOVIDA P/APPLE ZERO 350ML 12 S/W NP
2,1040,1271,COKE 500ML 12 S/W NP
3,10194,2062,SPRITE 500ML 20 RB
4,6540,5157,SCH NOVIDA PINEAPPLE BREEZ 300ML 24 RB
...,...,...,...
76,10170,5465,SCH TONIC 300ML 24 RB
77,7947,7477,RWENZORI WATER 500ML (24 BOX) NP
78,4765,11165,SPRITE ZERO 330ML 12 S/W NP
79,7534,10811,MM REFRESH MANGO 280ML 12 S/W


### Create Customers lookup table

In [942]:
# Create a new column 'Index' in both dataframes with current index values
top_customers['Index'] = top_customers.index
LE_Customers['Index'] = LE_Customers.index

In [943]:
if len(LE_Customers) != len(top_customers):
    raise SystemExit("Mismatch in number of customers")

In [944]:
LE_Customers = LE_Customers.drop_duplicates().copy()

In [945]:
# Merge based on Index
top_customers = pd.merge(
    top_customers,
    LE_Customers,
    how="left",
    on="Index",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [946]:
top_customers.drop(columns={'OrderLinesCount', 'Index'}, inplace=True, axis=1)

### Create Warehouses lookup table

In [947]:
LE_Warehouses

,D365_Del_Loc_LE,Index
0,MZ1-MZ010B,0
1,MZ1-MZ010Q,1
2,MZ1-MZ011B,2
3,MZ1-MZ011Q,3
4,MZ1-MZ012B,4
5,MZ1-MZ012Q,5
6,MZ1-MZ013B,6


In [948]:
top_warehouses

,D365_Del_Loc,OrderLinesCount,Grouping
0,ZA1-ZA014B,1113,0
1,ZA1-ZA016G,983,1
2,ZA1-ZA516B,951,2
3,ZA1-ZA020B,922,3
4,ZA1-ZA032B,921,4
...,...,...,...
82,ZA1-ZA034B,16,5
83,ZA1-ZA200B,13,6
84,ZA1-ZA565B,10,0
85,ZA1-ZA039B,5,1


In [949]:
# Create a new column 'Index' in both dataframes with current index values
#top_warehouses['Index'] = top_warehouses.index
LE_Warehouses['Index'] = LE_Warehouses.index

In [950]:
top_warehouses.rename(columns={'Grouping': 'Index'}, inplace=True)

In [951]:
#if len(LE_Warehouses) != len(top_warehouses):
#    raise SystemExit("Mismatch in number of warehouses")

In [952]:
# Merge based on Index
top_warehouses = pd.merge(
    top_warehouses,
    LE_Warehouses,
    how="left",
    on="Index",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [953]:
top_warehouses.drop(columns={'OrderLinesCount', 'Index'}, inplace=True, axis=1)

In [954]:
top_warehouses

,D365_Del_Loc,D365_Del_Loc_LE
0,ZA1-ZA014B,MZ1-MZ010B
1,ZA1-ZA550B,MZ1-MZ010B
2,ZA1-ZA505B,MZ1-MZ010B
3,ZA1-ZA024F,MZ1-MZ010B
4,ZA1-ZA525B,MZ1-MZ010B
...,...,...
82,ZA1-ZA052B,MZ1-MZ013B
83,ZA1-ZA544B,MZ1-MZ013B
84,ZA1-ZA511B,MZ1-MZ013B
85,ZA1-ZA527B,MZ1-MZ013B


### Merge Warehouses lookup table with SAP data

In [955]:
df5_1 = pd.merge(
    df5_W_C_P,
    top_warehouses,
    how="left",
    on="D365_Del_Loc",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [956]:
df5_W_C_P.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34396 entries, 919 to 870337
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ActivityType              34396 non-null  object
 1   Order Number              34396 non-null  int64 
 2   Material_Description      34396 non-null  object
 3   BaseDate                  34396 non-null  object
 4   BaseHour                  34396 non-null  int64 
 5   Cases                     34396 non-null  int64 
 6   Bill I                    34297 non-null  object
 7   Source Channel            34396 non-null  object
 8   Order Category            34396 non-null  object
 9   D365_ItemNo               34396 non-null  object
 10  Material No               34396 non-null  object
 11  MOD                       34396 non-null  int64 
 12  site_id                   34396 non-null  object
 13  WAREHOUSELOCATIONID       34396 non-null  object
 14  D365_Del_Loc            

In [957]:
df5_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34396 entries, 0 to 34395
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ActivityType              34396 non-null  object
 1   Order Number              34396 non-null  int64 
 2   Material_Description      34396 non-null  object
 3   BaseDate                  34396 non-null  object
 4   BaseHour                  34396 non-null  int64 
 5   Cases                     34396 non-null  int64 
 6   Bill I                    34297 non-null  object
 7   Source Channel            34396 non-null  object
 8   Order Category            34396 non-null  object
 9   D365_ItemNo               34396 non-null  object
 10  Material No               34396 non-null  object
 11  MOD                       34396 non-null  int64 
 12  site_id                   34396 non-null  object
 13  WAREHOUSELOCATIONID       34396 non-null  object
 14  D365_Del_Loc          

### Merge Products lookup table with SAP data

In [958]:
df5_2 = pd.merge(
    df5_1,
    top_products,
    how="left",
    on="D365_ItemNo",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [959]:
df5_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34396 entries, 0 to 34395
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ActivityType              34396 non-null  object
 1   Order Number              34396 non-null  int64 
 2   Material_Description      34396 non-null  object
 3   BaseDate                  34396 non-null  object
 4   BaseHour                  34396 non-null  int64 
 5   Cases                     34396 non-null  int64 
 6   Bill I                    34297 non-null  object
 7   Source Channel            34396 non-null  object
 8   Order Category            34396 non-null  object
 9   D365_ItemNo               34396 non-null  object
 10  Material No               34396 non-null  object
 11  MOD                       34396 non-null  int64 
 12  site_id                   34396 non-null  object
 13  WAREHOUSELOCATIONID       34396 non-null  object
 14  D365_Del_Loc          

### Merge Customers lookup table with SAP data

In [960]:
df5_3 = pd.merge(
    df5_2,
    top_customers,
    how="left",
    on="D365_Cust_No",
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
).copy()

In [961]:
df5_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34396 entries, 0 to 34395
Data columns (total 22 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ActivityType              34396 non-null  object
 1   Order Number              34396 non-null  int64 
 2   Material_Description      34396 non-null  object
 3   BaseDate                  34396 non-null  object
 4   BaseHour                  34396 non-null  int64 
 5   Cases                     34396 non-null  int64 
 6   Bill I                    34297 non-null  object
 7   Source Channel            34396 non-null  object
 8   Order Category            34396 non-null  object
 9   D365_ItemNo               34396 non-null  object
 10  Material No               34396 non-null  object
 11  MOD                       34396 non-null  int64 
 12  site_id                   34396 non-null  object
 13  WAREHOUSELOCATIONID       34396 non-null  object
 14  D365_Del_Loc          

In [962]:
df5_3.drop(columns={'site_id', 'D365_Cust_No', 'Material_Description', 'D365_ItemNo'}, inplace=True, axis=1)

In [963]:
df5_3.rename(columns={'site_id_LE': 'site_id', 'D365_Cust_No_LE': 'D365_Cust_No', 'Material_Description_LE': 'Material_Description', 'D365_ItemNo_LE': 'D365_ItemNo'}, inplace=True)

In [964]:
path = './data/'+ LE + '/15Dec_D365_orders_' + LE + '.feather'
df5_3.to_feather(path)

In [965]:
len(df5)

871832

In [970]:
len(df5["D365_Cust_No"].unique())

7022

In [966]:
len(df5_W)

871832

In [967]:
len(df5_W_C)

40017

In [968]:
len(df5_W_C_P)

34396

In [969]:
len(df5_W_C["D365_Cust_No"].unique())

170